In [1]:
import pandas as pd
from utils.aggregation_functions import (rellenar_etas, agrupar_descargas,
                                         estimar_demurrage, formato_BD)
from utils.extraction_functions import (extraer_bts, extraer_descargas, extraer_tiempos_de_viaje,
                   extraer_planificacion, extraer_programas, extraer_nueva_ficha, extraer_productos_plantas)

In [2]:
PATH_DISTANCIAS = "Distancias entre puertos.xlsx"
PATH_PROGRAMACION = "Programacion Descarga Importaciones 22 de Sep.xlsx"
PATH_NUEVA_FICHA = "Nueva Ficha Información de Buques.xlsx"
FECHA_PROGRAMACION = pd.to_datetime("2025-09-22")
FILE_NAME = f"Base de datos Estimación Semanal {FECHA_PROGRAMACION.strftime('%d-%m-%Y')}.xlsx"

In [3]:
df_bts = extraer_bts(PATH_PROGRAMACION, "Buques")
df_planificacion = extraer_planificacion(PATH_PROGRAMACION, "Planificación")
df_descargas = extraer_descargas(df_planificacion, ignore_not_bts=True, df_bts=df_bts)
df_programas = extraer_programas(df_planificacion)
df_productos_plantas = extraer_productos_plantas()
df_nueva_ficha = extraer_nueva_ficha(PATH_NUEVA_FICHA, "Programación de buques", df_programas=df_programas)
matriz_de_tiempos = extraer_tiempos_de_viaje("Distancias entre puertos.xlsx", "Datos")

df_descargas_productos_plantas = df_descargas.merge(df_productos_plantas, on=["Columna"]).drop(columns=["Columna"])
df_descargas_completo = df_descargas_productos_plantas.merge(df_bts, on=["Abrev."]).drop(columns=["Abrev."])
df_descargas_completo = df_descargas_completo[["Fecha", "N° Referencia", "Nombre programa", "Nombre del BT",
                                            "Producto", "Planta", "Ciudad", "Alias", "Volumen"]]

df_descargas_agrupadas = agrupar_descargas(df_descargas_completo)
df_descargas_agrupadas

c:\Users\jaubele.ap\AppData\Local\Programs\Python\Python313\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)
c:\Users\jaubele.ap\AppData\Local\Programs\Python\Python313\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


,Nombre programa,N° Referencia,Nombre del BT,Producto,Planta,Ciudad,Alias,Fecha inicio,Fecha fin,Volumen total,N° Descarga
1,102/25 Pintail Pacific,CC 102/25,Pintail Pacific,Gas 93,BT PUMA,NaN,Puma,2025-09-23 15:00:00,2025-09-23 23:00:00,15260,1
2,102/25 Pintail Pacific,CC 102/25,Pintail Pacific,Gas 93,TERMINAL TPI,Quintero,TPI,2025-09-25 15:00:00,2025-09-25 23:00:00,34000,2
3,103/25 Seaways Wheat /High Challenge,CC 103/25,Seaways Wheat /High Challenge,Gas 97,OXIQUIM QUINTERO,Quintero,Oxiquim Quintero,2025-10-23 15:00:00,2025-10-24 23:00:00,8000,1
4,103/25 Seaways Wheat /High Challenge,CC 103/25,Seaways Wheat /High Challenge,Gas 93,TERMINAL TPI,Quintero,TPI,2025-10-28 15:00:00,2025-10-29 23:00:00,40000,2
5,118/25 Andean Sun,CC 118/25,Andean Sun,Diesel A1,PLANTA MEJILLONES,Mejillones,Mejillones,2025-10-09 15:00:00,2025-10-10 23:00:00,30000,1
6,118/25 Andean Sun,CC 118/25,Andean Sun,Diesel A1,OXIQUIM CORONEL,Coronel,Coronel,2025-10-14 15:00:00,2025-10-15 23:00:00,18000,2
7,119/25 Seaways Dwarka,CC 119/25,Seaways Dwarka,Diesel A1,PLANTA IQUIQUE,Iquique,Iquique,2025-10-23 15:00:00,2025-10-24 23:00:00,12000,1
8,119/25 Seaways Dwarka,CC 119/25,Seaways Dwarka,Diesel A1,PLANTA MEJILLONES,Mejillones,Mejillones,2025-10-28 15:00:00,2025-10-29 23:00:00,36000,2
9,130/25 Seaways Stamford,CC 130/25,Seaways Stamford,Diesel A1,TERMINAL TPI,Quintero,TPI,2025-10-23 15:00:00,2025-10-24 23:00:00,32000,1
10,130/25 Seaways Stamford,CC 130/25,Seaways Stamford,Diesel A1,OXIQUIM CORONEL,Coronel,Coronel,2025-10-27 15:00:00,2025-10-28 23:00:00,15000,2


In [4]:
# Selecciona de la nueva ficha solo los programas de la programación
df_programas_completo = df_programas.merge(df_nueva_ficha, on="N° Referencia", how="left")

# Considera preferentemente las ventanas cortas, si no existen, las ventanas largas
df_programas_completo["Inicio Ventana"] = df_programas_completo["Inicio Ventana Corta"].combine_first(df_programas_completo["Inicio Ventana"])
df_programas_completo["Fin Ventana"] = df_programas_completo["Fin Ventana Corta"].combine_first(df_programas_completo["Fin Ventana"])

# Considera la ETA de la nueva ficha, si no existe, la ETA del programa.
df_programas_completo["ETA"] = df_programas_completo["ETA"].combine_first(df_programas_completo["ETA Programa"])

# Rellena los demurrages rates faltantes con la media
df_programas_completo["MONTO ($/DIA)"].fillna(df_programas_completo["MONTO ($/DIA)"].mean(), inplace=True)

df_programas_completo = df_programas_completo.drop(columns=["Inicio Ventana Corta", "Fin Ventana Corta", "ETA Programa"])
df_programas_completo.index = range(1, len(df_programas_completo) + 1)
df_programas_completo

C:\Users\jaubele.ap\AppData\Local\Temp\ipykernel_29096\648932889.py:12: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_programas_completo["MONTO ($/DIA)"].fillna(df_programas_completo["MONTO ($/DIA)"].mean(), inplace=True)


,N° Referencia,Proveedor,Inicio Ventana,Fin Ventana,MONTO ($/DIA),ETA
1,CC 75/25,PHILLIPS 66,2025-09-24,2025-09-26 23:59:59,32000.0,2025-09-25 11:00:00
2,CC 48/25,CHEVRON,2025-09-29,2025-10-01 23:59:59,40000.0,2025-09-29 00:00:00
3,CC 21/25,ARAMCO,2025-10-01,2025-10-03 23:59:59,34000.0,2025-10-04 15:00:00
4,CC 118/25,ARAMCO,2025-10-05,2025-10-07 23:59:59,33000.0,2025-10-06 00:00:00
5,CC 49/25,CHEVRON,2025-10-06,2025-10-08 23:59:59,40000.0,2025-10-06 20:00:00
6,CC 90/25,CHEVRON,2025-10-09,2025-10-11 23:59:59,40000.0,2025-10-10 02:00:00
7,CC 50/25,CHEVRON,2025-10-13,2025-10-17 23:59:59,33000.0,2025-10-14 02:00:00
8,CC 20/25,ARAMCO,2025-09-24,2025-09-26 23:59:59,34500.0,2025-09-29 12:00:00
9,CC 74/25,MARATHON,2025-10-14,2025-10-16 23:59:59,40000.0,2025-10-15 07:00:00
10,CC 130/25,ARAMCO,2025-10-14,2025-10-16 23:59:59,35000.0,2025-10-14 06:00:00


In [5]:
df_descargas_por_programa = df_descargas_agrupadas.merge(df_programas_completo, on="N° Referencia", how="right")
df_descargas_por_programa

,Nombre programa,N° Referencia,Nombre del BT,Producto,Planta,Ciudad,Alias,Fecha inicio,Fecha fin,Volumen total,N° Descarga,Proveedor,Inicio Ventana,Fin Ventana,MONTO ($/DIA),ETA
0,75/25 Flora Express,CC 75/25,Flora Express,Diesel A1,PLANTA IQUIQUE,Iquique,Iquique,2025-09-25 15:00:00,2025-09-26 23:00:00,8000,1,PHILLIPS 66,2025-09-24,2025-09-26 23:59:59,32000.0,2025-09-25 11:00:00
1,75/25 Flora Express,CC 75/25,Flora Express,Diesel A1,TERMINAL TPI,Quintero,TPI,2025-10-03 15:00:00,2025-10-04 23:00:00,42398,2,PHILLIPS 66,2025-09-24,2025-09-26 23:59:59,32000.0,2025-09-25 11:00:00
2,48/25 STI Mighty,CC 48/25,STI Mighty,Diesel A1,PLANTA IQUIQUE,Iquique,Iquique,2025-09-30 15:00:00,2025-10-01 23:00:00,12000,1,CHEVRON,2025-09-29,2025-10-01 23:59:59,40000.0,2025-09-29 00:00:00
3,48/25 STI Mighty,CC 48/25,STI Mighty,Diesel A1,PLANTA MEJILLONES,Mejillones,Mejillones,2025-10-04 15:00:00,2025-10-05 23:00:00,36000,2,CHEVRON,2025-09-29,2025-10-01 23:59:59,40000.0,2025-09-29 00:00:00
4,21/25 Seaways Athens,CC 21/25,Seaways Athens,Diesel A1,OXIQUIM QUINTERO,Quintero,Oxiquim Quintero,2025-10-04 15:00:00,2025-10-05 23:00:00,15000,1,ARAMCO,2025-10-01,2025-10-03 23:59:59,34000.0,2025-10-04 15:00:00
5,21/25 Seaways Athens,CC 21/25,Seaways Athens,Diesel A1,TERMINAL TPI,Quintero,TPI,2025-10-07 15:00:00,2025-10-08 23:00:00,33000,2,ARAMCO,2025-10-01,2025-10-03 23:59:59,34000.0,2025-10-04 15:00:00
6,118/25 Andean Sun,CC 118/25,Andean Sun,Diesel A1,PLANTA MEJILLONES,Mejillones,Mejillones,2025-10-09 15:00:00,2025-10-10 23:00:00,30000,1,ARAMCO,2025-10-05,2025-10-07 23:59:59,33000.0,2025-10-06 00:00:00
7,118/25 Andean Sun,CC 118/25,Andean Sun,Diesel A1,OXIQUIM CORONEL,Coronel,Coronel,2025-10-14 15:00:00,2025-10-15 23:00:00,18000,2,ARAMCO,2025-10-05,2025-10-07 23:59:59,33000.0,2025-10-06 00:00:00
8,49/25 Pacific Sentinel,CC 49/25,Pacific Sentinel,Diesel A1,PLANTA CALDERA,Caldera,Caldera,2025-10-09 15:00:00,2025-10-10 23:00:00,12000,1,CHEVRON,2025-10-06,2025-10-08 23:59:59,40000.0,2025-10-06 20:00:00
9,49/25 Pacific Sentinel,CC 49/25,Pacific Sentinel,Diesel A1,PLANTA MEJILLONES,Mejillones,Mejillones,2025-10-14 15:00:00,2025-10-15 23:00:00,36000,2,CHEVRON,2025-10-06,2025-10-08 23:59:59,40000.0,2025-10-06 20:00:00


In [6]:
df_descargas_por_programa["ETA"] = df_descargas_por_programa["ETA"][[True if descarga == 1 else False for descarga in df_descargas_por_programa["N° Descarga"]]]
rellenar_etas(df_descargas_por_programa, matriz_de_tiempos)

df_descargas_por_programa

,Nombre programa,N° Referencia,Nombre del BT,Producto,Planta,Ciudad,Alias,Fecha inicio,Fecha fin,Volumen total,N° Descarga,Proveedor,Inicio Ventana,Fin Ventana,MONTO ($/DIA),ETA
0,75/25 Flora Express,CC 75/25,Flora Express,Diesel A1,PLANTA IQUIQUE,Iquique,Iquique,2025-09-25 15:00:00,2025-09-26 23:00:00,8000,1,PHILLIPS 66,2025-09-24,2025-09-26 23:59:59,32000.0,2025-09-25 11:00:00
1,75/25 Flora Express,CC 75/25,Flora Express,Diesel A1,TERMINAL TPI,Quintero,TPI,2025-10-03 15:00:00,2025-10-04 23:00:00,42398,2,PHILLIPS 66,2025-09-24,2025-09-26 23:59:59,32000.0,2025-09-29 15:00:00
2,48/25 STI Mighty,CC 48/25,STI Mighty,Diesel A1,PLANTA IQUIQUE,Iquique,Iquique,2025-09-30 15:00:00,2025-10-01 23:00:00,12000,1,CHEVRON,2025-09-29,2025-10-01 23:59:59,40000.0,2025-09-29 00:00:00
3,48/25 STI Mighty,CC 48/25,STI Mighty,Diesel A1,PLANTA MEJILLONES,Mejillones,Mejillones,2025-10-04 15:00:00,2025-10-05 23:00:00,36000,2,CHEVRON,2025-09-29,2025-10-01 23:59:59,40000.0,2025-10-02 14:00:00
4,21/25 Seaways Athens,CC 21/25,Seaways Athens,Diesel A1,OXIQUIM QUINTERO,Quintero,Oxiquim Quintero,2025-10-04 15:00:00,2025-10-05 23:00:00,15000,1,ARAMCO,2025-10-01,2025-10-03 23:59:59,34000.0,2025-10-04 15:00:00
5,21/25 Seaways Athens,CC 21/25,Seaways Athens,Diesel A1,TERMINAL TPI,Quintero,TPI,2025-10-07 15:00:00,2025-10-08 23:00:00,33000,2,ARAMCO,2025-10-01,2025-10-03 23:59:59,34000.0,2025-10-05 23:00:00
6,118/25 Andean Sun,CC 118/25,Andean Sun,Diesel A1,PLANTA MEJILLONES,Mejillones,Mejillones,2025-10-09 15:00:00,2025-10-10 23:00:00,30000,1,ARAMCO,2025-10-05,2025-10-07 23:59:59,33000.0,2025-10-06 00:00:00
7,118/25 Andean Sun,CC 118/25,Andean Sun,Diesel A1,OXIQUIM CORONEL,Coronel,Coronel,2025-10-14 15:00:00,2025-10-15 23:00:00,18000,2,ARAMCO,2025-10-05,2025-10-07 23:59:59,33000.0,2025-10-14 01:00:00
8,49/25 Pacific Sentinel,CC 49/25,Pacific Sentinel,Diesel A1,PLANTA CALDERA,Caldera,Caldera,2025-10-09 15:00:00,2025-10-10 23:00:00,12000,1,CHEVRON,2025-10-06,2025-10-08 23:59:59,40000.0,2025-10-06 20:00:00
9,49/25 Pacific Sentinel,CC 49/25,Pacific Sentinel,Diesel A1,PLANTA MEJILLONES,Mejillones,Mejillones,2025-10-14 15:00:00,2025-10-15 23:00:00,36000,2,CHEVRON,2025-10-06,2025-10-08 23:59:59,40000.0,2025-10-11 21:00:00


## Parte 5: Calculo Demurrage/Laytime

In [7]:
df_estimacion = estimar_demurrage(df_descargas_por_programa)
df_estimacion

,Nombre programa,N° Referencia,Nombre del BT,Producto,Planta,Ciudad,Alias,Fecha inicio,Fecha fin,Volumen total,...,Inicio Ventana,Fin Ventana,MONTO ($/DIA),ETA,Inicio Laytime,Laytime descarga (Horas),Laytime programa (Horas),Laytime pactado (Horas),Demurrage programa (Horas),Estimación demurrage
0,75/25 Flora Express,CC 75/25,Flora Express,Diesel A1,PLANTA IQUIQUE,Iquique,Iquique,2025-09-25 15:00:00,2025-09-26 23:00:00,8000,...,2025-09-24,2025-09-26 23:59:59,32000.0,2025-09-25 11:00:00,2025-09-25 15:00:00,32.0,154.0,132,22.0,29334.0
1,75/25 Flora Express,CC 75/25,Flora Express,Diesel A1,TERMINAL TPI,Quintero,TPI,2025-10-03 15:00:00,2025-10-04 23:00:00,42398,...,2025-09-24,2025-09-26 23:59:59,32000.0,2025-09-29 15:00:00,2025-09-29 21:00:00,122.0,154.0,132,22.0,29334.0
2,48/25 STI Mighty,CC 48/25,STI Mighty,Diesel A1,PLANTA IQUIQUE,Iquique,Iquique,2025-09-30 15:00:00,2025-10-01 23:00:00,12000,...,2025-09-29,2025-10-01 23:59:59,40000.0,2025-09-29 00:00:00,2025-09-29 06:00:00,65.0,140.0,132,8.0,13334.0
3,48/25 STI Mighty,CC 48/25,STI Mighty,Diesel A1,PLANTA MEJILLONES,Mejillones,Mejillones,2025-10-04 15:00:00,2025-10-05 23:00:00,36000,...,2025-09-29,2025-10-01 23:59:59,40000.0,2025-10-02 14:00:00,2025-10-02 20:00:00,75.0,140.0,132,8.0,13334.0
4,21/25 Seaways Athens,CC 21/25,Seaways Athens,Diesel A1,OXIQUIM QUINTERO,Quintero,Oxiquim Quintero,2025-10-04 15:00:00,2025-10-05 23:00:00,15000,...,2025-10-01,2025-10-03 23:59:59,34000.0,2025-10-04 15:00:00,2025-10-04 15:00:00,32.0,98.0,132,0.0,0.0
5,21/25 Seaways Athens,CC 21/25,Seaways Athens,Diesel A1,TERMINAL TPI,Quintero,TPI,2025-10-07 15:00:00,2025-10-08 23:00:00,33000,...,2025-10-01,2025-10-03 23:59:59,34000.0,2025-10-05 23:00:00,2025-10-06 05:00:00,66.0,98.0,132,0.0,0.0
6,118/25 Andean Sun,CC 118/25,Andean Sun,Diesel A1,PLANTA MEJILLONES,Mejillones,Mejillones,2025-10-09 15:00:00,2025-10-10 23:00:00,30000,...,2025-10-05,2025-10-07 23:59:59,33000.0,2025-10-06 00:00:00,2025-10-06 06:00:00,113.0,153.0,132,21.0,28875.0
7,118/25 Andean Sun,CC 118/25,Andean Sun,Diesel A1,OXIQUIM CORONEL,Coronel,Coronel,2025-10-14 15:00:00,2025-10-15 23:00:00,18000,...,2025-10-05,2025-10-07 23:59:59,33000.0,2025-10-14 01:00:00,2025-10-14 07:00:00,40.0,153.0,132,21.0,28875.0
8,49/25 Pacific Sentinel,CC 49/25,Pacific Sentinel,Diesel A1,PLANTA CALDERA,Caldera,Caldera,2025-10-09 15:00:00,2025-10-10 23:00:00,12000,...,2025-10-06,2025-10-08 23:59:59,40000.0,2025-10-06 20:00:00,2025-10-07 02:00:00,93.0,185.0,132,53.0,88334.0
9,49/25 Pacific Sentinel,CC 49/25,Pacific Sentinel,Diesel A1,PLANTA MEJILLONES,Mejillones,Mejillones,2025-10-14 15:00:00,2025-10-15 23:00:00,36000,...,2025-10-06,2025-10-08 23:59:59,40000.0,2025-10-11 21:00:00,2025-10-12 03:00:00,92.0,185.0,132,53.0,88334.0


In [8]:
df_BD = formato_BD(df_estimacion, df_descargas_completo, FECHA_PROGRAMACION)
df_BD

,Fecha de programación,Semana,Año,Mes,Horas Laytime,CC,Nombre BT,Proveedor,Producto,Demurrage,Puerto,Volumen,Inicio Ventana,Final Ventana,ETA,Fin descarga,Estimación demurrage
0,22-09-2025,Semana 4,2025,Octubre,132,CC 75/25,Flora Express,PHILLIPS 66,Diesel A1,32000.0,Iquique,8000,24-09-2025,26-09-2025,25-09-2025 11:00,26-09-2025,29334.0
1,22-09-2025,Semana 4,2025,Octubre,132,CC 75/25,Flora Express,PHILLIPS 66,Diesel A1,32000.0,TPI,42398,24-09-2025,26-09-2025,29-09-2025 15:00,04-10-2025,29334.0
2,22-09-2025,Semana 4,2025,Octubre,132,CC 48/25,STI Mighty,CHEVRON,Diesel A1,40000.0,Iquique,12000,29-09-2025,01-10-2025,29-09-2025 00:00,01-10-2025,13334.0
3,22-09-2025,Semana 4,2025,Octubre,132,CC 48/25,STI Mighty,CHEVRON,Diesel A1,40000.0,Mejillones,36000,29-09-2025,01-10-2025,02-10-2025 14:00,05-10-2025,13334.0
4,22-09-2025,Semana 4,2025,Octubre,132,CC 21/25,Seaways Athens,ARAMCO,Diesel A1,34000.0,Oxiquim Quintero,15000,01-10-2025,03-10-2025,04-10-2025 15:00,05-10-2025,0.0
5,22-09-2025,Semana 4,2025,Octubre,132,CC 21/25,Seaways Athens,ARAMCO,Diesel A1,34000.0,TPI,33000,01-10-2025,03-10-2025,05-10-2025 23:00,08-10-2025,0.0
6,22-09-2025,Semana 4,2025,Octubre,132,CC 118/25,Andean Sun,ARAMCO,Diesel A1,33000.0,Mejillones,30000,05-10-2025,07-10-2025,06-10-2025 00:00,10-10-2025,28875.0
7,22-09-2025,Semana 4,2025,Octubre,132,CC 118/25,Andean Sun,ARAMCO,Diesel A1,33000.0,Coronel,18000,05-10-2025,07-10-2025,14-10-2025 01:00,15-10-2025,28875.0
8,22-09-2025,Semana 4,2025,Octubre,132,CC 49/25,Pacific Sentinel,CHEVRON,Diesel A1,40000.0,Caldera,12000,06-10-2025,08-10-2025,06-10-2025 20:00,10-10-2025,88334.0
9,22-09-2025,Semana 4,2025,Octubre,132,CC 49/25,Pacific Sentinel,CHEVRON,Diesel A1,40000.0,Mejillones,36000,06-10-2025,08-10-2025,11-10-2025 21:00,15-10-2025,88334.0
